In [1]:
from bs4 import BeautifulSoup
from amazon_sel import Amazon
import requests

import sys
print(sys.executable)   # just to check we are in correct env or not !!!

C:\Users\sarat\miniconda3\envs\agg\python.exe


In [2]:
def extract_product_divs(html):
    # these are different layouts of amazon product divs
    different_web_ele_selectors = [
        'div.sg-col-4-of-12.s-result-item.s-asin.sg-col-4-of-16.sg-col.sg-col-4-of-20',
        'div.s-result-item.s-asin.sg-col-0-of-12.sg-col-16-of-20.sg-col.sg-col-12-of-16'
    ]

    for pattern in different_web_ele_selectors:
        product_divs = html.select(pattern)

        if len(product_divs) > 0:
            break  # it means page is grid. No need to check for other alignment

    return product_divs

In [3]:

def extract_images(product_divs) :
    
    # just a wrapper to iterate each product and extract img-link
    

    for i in range(len(product_divs)) :   # for each product
        uid = product_divs[i].get('data-asin')    # for any product, this unique id will be present...
        try :

            prod = product_divs[i]  
            
            img_link = prod.find('img').get('src')

            download_img(img_link, uid)   # download img and saves the image

        except :
            print(uid)   # products for which image is failed to download
            continue
            
    
def download_img(img_url, uid) :
    # it will store elements in the images folder. make sure it already exists !!!
    img = requests.get(img_url).content
    
    with open(f'images/{uid}.jpg', 'wb') as f :
        f.write(img)
        

    

In [4]:
# we ask user to enter the product to be searched, and number of items or pages
def input_details() : 

    print('\nEnter the Details\n')
    item_name = input('Enter the name of product to be searched : ')

    if not item_name :
        # without product name, we cannot scrape anything...
        return -1

    num_items = input('Enter the number of products to be processed (default: 100)')
    num_pages = input('Enter the number of pages to be traversed. (default: 10)')


    if not num_items :
        num_items = 100  # default
        
    else :
        num_items = int(num_items)   # string to int

    if not num_pages :
        num_pages = 10  # default
        
    else :
        num_pages = int(num_pages)

    print('-'* 50)
    print('Entered Details : \n')
    print(f'Item  :\t {item_name}')
    print(f'Number of items to be processed  :\t {num_items}')
    print(f'Number of pages to be processed  :\t {num_pages}')
    
    print('-'* 50)
    
    
    return item_name, num_items, num_pages


### Open Amazon and search for the product

In [12]:
def print_details(count, pages, completed=False) :
    
    if not completed :
        if (count % 25 == 0) or (pages % 5 == 0) :

            print('='* 50)
            print(f'\nProducts extracted: {count}\nPages Traversed : {pages}\n')
            print('=' * 50)
            
    else :
        # prints for the last time !!
        print('='* 30+' Final Details '+ '='* 20)
        print(f'\nTotal Number of Products extracted: {count}\nTatal Pages Traversed : {pages}\n')
        print('=' * 50)



# take the input details
# iterate till one of condition met !!

details = input_details()

if details == -1 :
    sys.exit('Exiting Since product name is not entered !!')
    
item_name, num_prods, num_pages = details[0], details[1], details[2]


ama_search = Amazon()  # instantiate amazon search class
pg_src = ama_search.search_text(item_name)

count_products = 0
pages_traversed = 1   # already 1 page is opened !!!

# break when required products count is reached or req of pages are traversed !!!

while (count_products < num_prods) and (pages_traversed < num_pages ):  
    
    html = BeautifulSoup(pg_src, 'lxml')
    product_divs = extract_product_divs(html)  # extract the actual product divs
    
    extract_images(product_divs)  # will download all the images of extracted products
    
    # Inc the prods count
    count_products += len(product_divs)
    
    
    pg_src = ama_search.next_page()   # open next page....
    
    
    if pg_src == -1 :
        print('Extraction Completed !!')
        break
    
    pages_traversed += 1
    
    print_details(count_products, pages_traversed)   # helper func to print details
    
    
print_details(count_products, pages_traversed, True)  # print all the details 
    
    


Enter the Details



Enter the name of product to be searched :  tomato
Enter the number of products to be processed (default: 100) 200
Enter the number of pages to be traversed. (default: 10) 


--------------------------------------------------
Entered Details : 

Item  :	 tomato
Number of items to be processed  :	 200
Number of pages to be processed  :	 10
--------------------------------------------------

Products extracted: 240
Pages Traversed : 5

============================== Final Details ====================

Total Number of Products extracted: 240
Tatal Pages Traversed : 5

